In [1]:
import glob
import os
import py3Dmol
import math
import pandas as pd
import numpy as np
import matplotlib
import copy
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

import itertools
import sys
sys.path.append('../')
from mypackage.AnalyzeGeom import cpx
from mypackage.StructGen import structgen
from mypackage.globalvars import MYPACKAGE_DIR
from mypackage import visualize as vis
from mypackage.AnalyzeGeom import geommath



Hartree = 627.503


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%reload_ext autoreload

In [243]:
files = glob.glob('../ThesisFiles/backup_fulloutput/TSOA/*/*.log')

# XYZ files

In [3]:
def ligand_breakdown(mol,dummy_bool=False):
    # get networkx graph representations
    g=mol3D_to_networkx(mol,get_symbols=False,get_bond_order=False,get_bond_distance=False)
    
    # get metal_indices
    metal_indices=mol.findMetal(transition_metals_only=True)
    
    # init ca dict
    met_ca_d = {}
    for met in sorted(metal_indices):
        met_sym = mol.atoms[met].sym
        met_ca_d[met] = {"symbol":met_sym,"catoms":[]}        
        
    # get all connecting atoms
    bonds = g.edges()
    for bond in bonds:
        at0 = bond[0]
        at1 = bond[1]
        if at0 in met_ca_d:
            met_ca_d[at0]["catoms"].append(at1)
        elif at1 in metal_indices:
            met_ca_d[at1]["catoms"].append(at0)
    
    connecting_atoms = []
    
    # remove all metals from copied graph and get all disconnected graphs from the copy
    g.remove_nodes_from(metal_indices)
    disconnected_graphs = list(nx.connected_components(g))
    
    list_of_ligands=[]
    # for each disconnected graph
    for set_of_nodes in disconnected_graphs:
        metal_indices=mol.findMetal(transition_metals_only=True)
        bridging_data=bridging_ligand_helper(mol,set_of_nodes,metal_indices)
        listed_nodes =list(set_of_nodes)
        if dummy_bool:
            submol=mol.create_mol_with_inds(listed_nodes+metal_indices)
            # get metal_indices
            metal_indices=submol.findMetal(transition_metals_only=True)
            for metal_index in metal_indices:
                submol.getAtom(metal_index).mutate(newType='X')
        else:
            # create a mol3D of just the atoms in the subgraph and get the new mol2
            submol=mol.create_mol_with_inds(listed_nodes)   
            
        # connecting atoms cont
        bridging_data['catoms'] = {}
        for met in met_ca_d:
            sub_connecting_atoms = []
            connecting_atoms = met_ca_d[met]["catoms"]
            for node in listed_nodes:
                if node in connecting_atoms:
                    sub_connecting_atoms.append(listed_nodes.index(node))
            name = f"{mol.atoms[met].sym}_0"
            while name in bridging_data['catoms']:
                inc = int(name.split('_')[1])
                name = f"{met_sym}_{inc+1}"
            bridging_data['catoms'][name]=sub_connecting_atoms
        
        # graph hash
        g_sub=mol3D_to_networkx(submol,get_symbols=True,get_bond_order=False,get_bond_distance=False)
        gh = nx.weisfeiler_lehman_graph_hash(g_sub, node_attr = 'symbol')
        bridging_data['graph_hash'] = gh
        # add to list
        list_of_ligands.append((submol,bridging_data))
    return list_of_ligands
def mol3D_to_networkx(mol,get_symbols:bool=True,get_bond_order:bool=True,get_bond_distance:bool=False):
    g = nx.Graph()
    
    # get every index of atoms in mol3D object
    for atom_ind in range(0,mol.natoms):
        # set each atom as a node in the graph, and add symbol information if wanted
        data={}
        if get_symbols:
            data['symbol']=mol.getAtom(atom_ind).symbol()
        g.add_node(atom_ind,**data)
        
    # get every bond in mol3D object
    bond_info=mol.bo_dict
    for bond in bond_info:
        # set each bond as an edge in the graph, and add symbol information if wanted
        data={}
        if get_bond_order:
            data['bond_order']=bond_info[bond]
        if get_bond_distance:
            distance=mol.getAtom(bond[0]).distance(mol.getAtom(bond[1]))
            data['bond_distance']=distance
        g.add_edge(bond[0],bond[1],**data)
    return g
def bridging_ligand_helper(mol,set_of_nodes,metal_indices):
    submol=mol.create_mol_with_inds(list(set_of_nodes)+metal_indices)
    # get new metal indicies
    submol_metal_indices=submol.findMetal(transition_metals_only=True)
    # get disconnected graphs
    submol.nx_graph = mol3D_to_networkx(submol, True, False, False)
    disconnected_graphs = list(nx.connected_components(submol.nx_graph))
    # check if at least two metals present in each disconnected graph
    ret={}
    for d_graph in disconnected_graphs:
        # if length is 1 then it is more than isolated metal
        if len(d_graph) > 1:
            # count each metal, see if they are in metal indices
            metal_count=0
            for node in d_graph:
                if node in submol_metal_indices:
                    metal_count+=1
            ret['Bridging_Ligand']=metal_count>1
            ret['Connected_Metals']=metal_count
    return ret

In [40]:
xyzfile = '../ThesisFiles/file_xyz/I1/I1_Ni_SIPr_monolig_optfreq.xyz'

In [42]:
def normalize(vector):
    """ Normalize a vector to have a unit length. """
    return vector / np.linalg.norm(vector)

def angle_between_vectors(v1, v2):
    """Calculate the angle in radians between two vectors."""
    v1_u = normalize(v1)
    v2_u = normalize(v2)
    dot_product = np.clip(np.dot(v1_u, v2_u), -1.0, 1.0)  # Ensure within [-1, 1]
    return np.arccos(dot_product)

def rotation_matrix(axis, angle):
    """
    Create a rotation matrix for rotating around a given axis by a specified angle.
    
    Parameters:
    - axis: A 3-element array specifying the axis of rotation.
    - angle: The angle in radians by which to rotate.
    
    Returns:
    - A 3x3 rotation matrix.
    """
    axis = normalize(axis)
    a = np.cos(angle / 2.0)
    b, c, d = -axis * np.sin(angle / 2.0)
    return np.array([
        [a*a + b*b - c*c - d*d, 2*(b*c + a*d), 2*(b*d - a*c)],
        [2*(b*c - a*d), a*a + c*c - b*b - d*d, 2*(c*d + a*b)],
        [2*(b*d + a*c), 2*(c*d - a*b), a*a + d*d - b*b - c*c]
    ])

def rotate_coordinates(coords, center, rotation_matrix):
    """
    Rotate a set of coordinates around a specified center using a rotation matrix.
    
    Parameters:
    - coords: An Nx3 array of coordinates to rotate.
    - center: The center of rotation as a 3-element array.
    - rotation_matrix: The 3x3 rotation matrix.
    
    Returns:
    - The rotated Nx3 array of coordinates.
    """
    # Translate coordinates to origin
    translated_coords = coords - center
    # Apply the rotation
    rotated_coords = np.dot(translated_coords, rotation_matrix.T)
    # Translate back to original position
    final_coords = rotated_coords + center
    return final_coords

# polish adding ligand code

1. Get Mol template

In [172]:
template_Mol = structgen.create_template_Mol()
num_atoms = template_Mol.natoms
atoms_template = template_Mol.atoms
template_Mol.coords

array([[ 0. ,  0. ,  0. ],
       [ 1.8,  0. ,  0. ],
       [ 0. ,  1.8,  0. ],
       [-1.8,  0. ,  0. ],
       [ 0. , -1.8,  0. ]])

2. Define metal center

In [173]:
metal_center = ['Pd'] # input: list of metal (either mononuclear, dinuclear)

atoms_template_M = [atom for atom in atoms_template if atom.sym == 'M']
if len(metal_center) < len(atoms_template_M):
    raise ValueError("Not enough elements in metal_center to match the atoms with symbol 'M'.")
for idx, atom in enumerate(atoms_template_M):
    atom.sym = metal_center[idx % len(metal_center)]

atoms_template

[Atom(Pd0), Atom(X1), Atom(X2), Atom(X3), Atom(X4)]

2. get ligand Mol

In [174]:
phosphene_path = '../23_genstruct/04_myligands/LPH001.mol2' # input: ligand structure
lig_Mol = cpx.MolLigand()
lig_Mol.readfile('mol2', phosphene_path)
catoms = [0] # input: catoms
lig_Mol.set_catoms(catoms)
lig_Mol.getAtom_catoms()

{Atom(P0)}

3. get M-L bond length

In [175]:
metal_sym = atoms_template[0].sym
ligand_sym = list(lig_Mol.getAtom_catoms())[0].sym # monodentate ligand
ML_bondlength = 2.26 # for Pd-P

4. scale the M-L distance

In [187]:
pointM = atoms_template[0].coord
pointL = atoms_template[2].coord # iterate for other points
pointL = geommath.scaled_distance(pointM, pointL, ML_bondlength)
pointL

array([0.  , 2.26, 0.  ])

4. Translate ligand to the connecting point

In [188]:
catom = list(lig_Mol.getAtom_catoms())[0] # monodentate ligand
translated_vector = pointL - catom.coord
translated_coord = lig_Mol.coords + translated_vector

5. find lone pair vector

In [189]:
adjcatoms = lig_Mol.getAtom_adjcatom()[catoms[0]] # monodentate ligand
adjcatom_coords = lig_Mol.getcoord_fromatomlist(adjcatoms)
adjcatom_vecs = adjcatom_coords - catom.coord
lonepair_vec = geommath.normalize(-np.sum(adjcatom_vecs, axis=0))
lonepair_vec

array([-0.730371  , -0.09521714,  0.67638147])

6. find bonding vector

In [190]:
templatebond_vec = geommath.normalize(pointM - pointL)
templatebond_vec

array([ 0., -1.,  0.])

7. rotate ligands to bond the template

In [191]:
# Calculate rotation axis and angle
rotation_axis = np.cross(lonepair_vec, templatebond_vec)
rotation_angle = angle_between_vectors(lonepair_vec, templatebond_vec)
# Create rotation matrix
R = rotation_matrix(rotation_axis, rotation_angle)
# Apply rotation to the coordinates centered at catom_coord
rotated_coords = rotate_coordinates(translated_coord, pointL, R)

8. Add ligands to the template complex

In [192]:
list_sym = lig_Mol.getlistofsym()

In [193]:
for sym, coord in zip(list_sym, rotated_coords):
    num_atoms = template_Mol.natoms
    template_Mol.addAtom(cpx.Atom(num_atoms,sym,coord))
template_Mol

Mol(natoms=69)

In [194]:
template_Mol.deleteAtoms_bysym(['X'])

ValueError: No atoms with symbols ['X'] found.

In [195]:
vis.showxyz_fromxyzstr(template_Mol.writexyz())

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [222]:
xyzfiles = glob.glob('../ThesisFiles/file_xyz/TSOA/*')
xyzfile = xyzfiles[2]
vis.showxyz_fromxyzfile(xyzfile)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [223]:
mycpx = cpx.Mol()
mycpx.readfile('xyz', xyzfile)
mycpx.deleteAtoms(list(range(15, 39)))
mycpx.atoms

[Atom(P0),
 Atom(Ni1),
 Atom(P2),
 Atom(Br3),
 Atom(C4),
 Atom(C5),
 Atom(C6),
 Atom(C7),
 Atom(H8),
 Atom(C9),
 Atom(H10),
 Atom(C11),
 Atom(H12),
 Atom(H13),
 Atom(H14)]

In [224]:
vis.showxyz_fromxyzstr(mycpx.writexyz())

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [227]:
coords = mycpx.coords
coord_metal = mycpx.get_coord_byidx(1)
coords - coord_metal


array([[-1.87661269,  1.16264594,  0.09867081],
       [ 0.        ,  0.        ,  0.        ],
       [ 1.79756385,  1.21063879, -0.29172262],
       [-0.20051184, -1.81710838, -1.70084032],
       [ 0.68044637, -1.71956036,  0.23407793],
       [ 1.98680838, -2.23870452,  0.39584541],
       [-0.27602551, -1.85768445,  1.27382688],
       [ 2.38144963, -2.67207013,  1.64596561],
       [ 2.67340276, -2.25689664, -0.44548871],
       [ 0.17264075, -2.27458406,  2.54372419],
       [-1.34025153, -1.81287107,  1.0674139 ],
       [ 1.48797867, -2.6595717 ,  2.73853158],
       [ 3.40107109, -3.02469631,  1.78861163],
       [-0.54567432, -2.36038828,  3.35698728],
       [ 1.81867318, -3.01085117,  3.71226826]])

In [231]:
structgen.create_template_Mol('tsoa')

Mol(natoms=15)